# 風実装のためのテストプログラム

ライブラリ読み込み

In [126]:
import polars as pl
import numpy as np
from geopy.distance import geodesic
from geopy.point import Point
import math

# 初期方位角を計算する関数
from geographiclib.geodesic import Geodesic

風データの読み込み

In [127]:
year = 2016
month = 4

wind_data = pl.read_csv("ERA5/wind_datas/era5_testdata_E180W90S0W90_" + str(int(year)) + "_" + str(int(month)) + ".csv",encoding="shift-jis")

計算

In [167]:
# 風速情報と点Sの移動軌跡に最も近い地点を見つける関数
def find_nearest_wind_point(data, lat, lon):
    # 最も近い風速情報を取得
    nearest_point = data.with_columns(
        (pl.lit(lat) - pl.col('LAT')).abs().alias('lat_diff'),
        (pl.lit(lon) - pl.col('LON')).abs().alias('lon_diff')
    ).with_columns(
        (pl.col('lat_diff') + pl.col('lon_diff')).alias('total_diff')
    ).sort('total_diff').head(1)
    
    # 最も近い風速情報を取得
    u = nearest_point["U10_E+_W-[m/s]"][0]
    v = nearest_point["V10_N+_S-[m/s]"][0]
    
    #print("現在地から1番近いポイント＝",nearest_point)

    return u, v

# 点Sが風から得るエネルギーを計算する関数
def calculate_energy(u, v, angle_difference, time):
    # 風速と角度差を考慮してエネルギーを計算
    if abs(angle_difference) <= 10 or abs(angle_difference) >= 350:
        # 後方からの追い風または横風
        energy = (abs(u) + abs(v)) * time
    elif abs(angle_difference) < 90 or abs(angle_difference) > 270:
        # 横風
        energy = (abs(u) + abs(v)) * time
    else:
        # 向かい風
        energy = - (abs(u) + abs(v)) * time
    
    return energy



def calculate_initial_bearing(start_coord, end_coord):
    # Geodesicクラスのインスタンスを作成
    geod = Geodesic.WGS84

    
    # 始点と終点の緯度と経度
    lat1, lon1 = start_coord[0], start_coord[1]
    lat2, lon2 = end_coord[0], end_coord[1]
    
    # geod.Inverse()を使用して始点から終点までの地理的な情報を取得
    # 結果として辞書が返され、辞書のキー 'azi1' は初期方位角を含む
    result = geod.Inverse(lat1, lon1, lat2, lon2)
    
    # 初期方位角を取得（北を0度とし、時計回りに測定）
    initial_bearing = result['azi1']
    
    return initial_bearing


# 移動軌跡と風速情報からエネルギーを計算するプログラム
def calculate_trajectory_energy(data, start_coord, end_coord, timestep):

    # 初期方位角を計算
    initial_bearing = calculate_initial_bearing(start_coord, end_coord)
    
    # 始点と終点を Point オブジェクトに変換
    start_point = Point(start_coord)
    end_point = Point(end_coord)
    print("start_point",start_point)
    print("end_point",end_point)
    
    # 大圏距離を計算
    geo_dist = geodesic(start_point, end_point)
    total_distance = geo_dist.kilometers
    
    # 時間間隔の設定
    time_interval = timestep / (2 * timestep)
    
    # 初期化
    current_position = start_point
    current_bearing = initial_bearing
    total_energy = 0
    
    # 時間間隔ごとの移動
    for i in range(int(2 * timestep)):
        # 現在の時刻
        time = i * time_interval
        
        # 指定された時間での移動距離を計算
        distance_travelled = (total_distance / timestep) * time
        
        # 現在の位置を計算
        current_position = geodesic(kilometers=distance_travelled).destination(
            point=start_point, bearing=current_bearing
        )
        
        # 現在の方位角を更新
        current_bearing = calculate_initial_bearing(current_position, end_coord)
        
        # 現在の位置の緯度経度
        current_lat, current_lon = current_position.latitude, current_position.longitude
        
        # 風速データを取得
        # ここでは風速データを取得するための関数 find_nearest_wind_point を使用します。
        # この関数は data から緯度経度に最も近い風速データを取得するものです。
        u, v = find_nearest_wind_point(data, current_lat, current_lon)
        
        # 風向角度を計算（北を 0 度として時計回りに増加）
        wind_angle = math.degrees(math.atan2(v, u))
        # 北を 0 度として時計回りに測定するように調整
        wind_angle = (360 - wind_angle + 90) % 360 
        
        # 船の方位角と風向の角度差を計算
        if current_bearing < 0:
            current_bearing += 360
        
        # 船の方位から見て風がどの方向に向かって吹いているのかを計算している。（必要に応じて吹いて来ている方向に直す）
        angle_difference = (wind_angle - current_bearing + 360) % 360
        
        # エネルギーを計算
        energy = calculate_energy(u, v, angle_difference, time_interval)
        
        # エネルギーを累積
        total_energy += energy

        #print("座標",current_lat,current_lon)
        #print("風速",u,v)
        print("進路",current_bearing)
        print("風向",wind_angle)
        print("角度差",angle_difference)

    
    return total_energy

# 開始座標、終了座標、時間間隔
# 始点と終点の座標
start_coord = (40.0, 135.0)
end_coord = (41.0, 134.0)
timestep = 6

# プログラムを実行してエネルギーを計算
data = wind_data
total_energy = calculate_trajectory_energy(data, start_coord, end_coord, timestep)

print(end_coord)

#print(f'点Sが風から得たエネルギー: {total_energy}')


start_point 40 0m 0s N, 135 0m 0s E
end_point 41 0m 0s N, 134 0m 0s E
進路 322.96670009824686
風向 41.453845564545475
角度差 78.48714546629861
進路 322.9138119374266
風向 41.453845564545475
角度差 78.54003362711887
進路 322.87114888771856
風向 39.47753385271744
角度差 76.60638496499888
進路 322.83886503838187
風向 39.47753385271744
角度差 76.63866881433557
進路 322.81709695176886
風向 39.47753385271744
角度差 76.66043690094858
進路 322.8059630200572
風向 39.12445728214141
角度差 76.31849426208419
進路 322.80556291699884
風向 39.12445728214141
角度差 76.31889436514257
進路 322.8159771488418
風向 39.12445728214141
角度差 76.30848013329961
進路 322.83726670692977
風向 39.81436403305713
角度差 76.97709732612736
進路 322.8694728228888
風向 39.81436403305713
角度差 76.94489121016835
進路 322.91261682559605
風向 39.81436403305713
角度差 76.90174720746109
進路 322.96670009855717
風向 41.792274602757686
角度差 78.82557450420052
(41.0, 134.0)
